In [1]:
import glob
import cv2
import numpy as np
import os
import traceback

In [3]:

def detect_and_remove_logo(image, logo_templates, threshold=0.8):
    """
    Detects and removes logos from an image using template matching + inpainting.
    
    Parameters:
        image: Input image (BGR)
        logo_templates: List of logo template images (BGR)
        threshold: Match quality threshold (0–1, higher = stricter)
    
    Returns:
        Cleaned image with logos removed if detected.
    """
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    for template in templates:
        if template is None:
            continue  # Skip if template failed to load

        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

        # Skip if template is larger than image
        if template_gray.shape[0] > image_gray.shape[0] or template_gray.shape[1] > image_gray.shape[1]:
            continue

        res = cv2.matchTemplate(image_gray, template_gray, cv2.TM_CCOEFF_NORMED)
        loc = np.where(res >= threshold)

        for pt in zip(*loc[::-1]):
            h, w = template_gray.shape
            cv2.rectangle(image, pt, (pt[0] + w, pt[1] + h), (255, 255, 255), thickness=cv2.FILLED)

    return image



In [4]:
import cv2
import numpy as np

def extract_irregular_coin_safe(image, target_size=(224, 224), margin=10, debug=False):
    """
    Safely extracts irregular or partial coins using robust contour detection.
    Keeps aspect ratio, adds transparency padding.
    """
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply CLAHE for local contrast boost
    clahe = cv2.createCLAHE(clipLimit=20.0, tileGridSize=(16, 16))
    equalized = clahe.apply(gray)

    # Then blur to reduce noise
    blurred = cv2.GaussianBlur(equalized, (5, 5), 0)
    thresh = cv2.adaptiveThreshold(
        blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV, 9,2
    )


    kernel = np.ones((3, 3), np.uint8)
    cleaned = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

    # Find contours
    contours, _ = cv2.findContours(cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        print("No valid contours found.")
        return None

    largest = max(contours, key=cv2.contourArea)
    area = cv2.contourArea(largest)
    if area < 100:  # ignore very small artifacts
        print("Detected contour is too small.")
        return None

    mask = np.zeros_like(gray)
    cv2.drawContours(mask, [largest], -1, 255, thickness=cv2.FILLED)

    coin_only = cv2.bitwise_and(image, image, mask=mask)

    # crop to bounding box
    x, y, w, h = cv2.boundingRect(largest)
    x1 = max(x - margin, 0)
    y1 = max(y - margin, 0)
    x2 = min(x + w + margin, image.shape[1])
    y2 = min(y + h + margin, image.shape[0])

    coin_crop = coin_only[y1:y2, x1:x2]
    mask_crop = mask[y1:y2, x1:x2]

    coin_bgra = cv2.cvtColor(coin_crop, cv2.COLOR_BGR2BGRA)
    coin_bgra[:, :, 3] = mask_crop

    # keep ratio
    h, w = coin_bgra.shape[:2]
    scale = min(target_size[0] / h, target_size[1] / w)
    new_w, new_h = int(w * scale), int(h * scale)
    resized = cv2.resize(coin_bgra, (new_w, new_h), interpolation=cv2.INTER_AREA)

    final = np.zeros((target_size[1], target_size[0], 4), dtype=np.uint8)
    x_offset = (target_size[0] - new_w) // 2
    y_offset = (target_size[1] - new_h) // 2
    final[y_offset:y_offset + new_h, x_offset:x_offset + new_w] = resized

    if debug:
        cv2.imwrite("debug_mask.png", mask)
        cv2.imwrite("debug_thresh.png", thresh)
        cv2.imwrite("debug_cleaned.png", cleaned)

        
    return final


In [4]:
image_paths = glob.glob("../bueschelquinare/*/*/*.jpg")

template1 = cv2.imread("muenzkabinett_berlin.png")
template2 = cv2.imread("cc-by-nc-sa.png")
templates = [template1, template2]

for image_path in image_paths:
    img = cv2.imread(image_path)
    try:
        cleaned_img = detect_and_remove_logo(img, templates)
        extraced_coin_img = extract_irregular_coin_safe(cleaned_img)
    except:
        print(f"Error: {image_path}")
        traceback.print_exc()

    # extract alpha mask
    alpha = extraced_coin_img[:, :, 3]
    foreground = extraced_coin_img[:, :, :3]
    masked = cv2.bitwise_and(foreground, foreground, mask=alpha)
    gray_masked = cv2.cvtColor(masked, cv2.COLOR_BGR2GRAY)
    gray_rgb = cv2.cvtColor(gray_masked, cv2.COLOR_GRAY2BGR)
    gray_bgra = cv2.merge([gray_rgb, alpha])
    image_path_to_save = image_path.replace("bueschelquinare","bueschelquinare_preprocessed")
    image_path_to_save = image_path_to_save.replace(".jpg","_cropped_graymasked.png")
    if extraced_coin_img is not None:
        cv2.imwrite(image_path_to_save, gray_bgra)
